# 🎓 Comparaison des optimiseurs SGD et Adam

Ce notebook compare deux algorithmes d'optimisation populaires en Deep Learning :
- **SGD** (Stochastic Gradient Descent) avec momentum
- **Adam** (Adaptive Moment Estimation)

Nous allons les implémenter de A à Z pour bien comprendre leur fonctionnement !

## 📦 Imports nécessaires

On importe PyTorch pour créer notre réseau de neurones et matplotlib pour visualiser les résultats.

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

## 🔧 Implémentation de SGD personnalisé

**SGD (Stochastic Gradient Descent)** est l'optimiseur de base :
- Il met à jour les poids en suivant la direction opposée du gradient
- Le **momentum** permet d'accélérer la convergence en gardant une "mémoire" des directions précédentes

**Formule** : `v = momentum × v + gradient` puis `poids = poids - learning_rate × v`

In [ ]:
class SGDPerso:
    def __init__(self, params, lr=0.01, momentum=0.0):
        """Initialise l'optimiseur SGD.
        
        Args:
            params: Les paramètres du modèle à optimiser
            lr: Taux d'apprentissage (learning rate)
            momentum: Coefficient de momentum (0 = pas de momentum)
        """
        self.params = list(params)
        self.lr = lr
        self.momentum = momentum
        # v stocke la "vitesse" pour chaque paramètre (utilisé pour le momentum)
        self.v = [torch.zeros_like(p.data) for p in self.params]

    def zero_grad(self):
        """Remet à zéro tous les gradients avant le backward."""
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        """Effectue une étape d'optimisation en mettant à jour les poids."""
        for i, p in enumerate(self.params):
            if p.grad is None:
                continue
            g = p.grad.data  # Récupère le gradient
            # Mise à jour de la vitesse avec momentum
            self.v[i] = self.momentum * self.v[i] + g
            # Mise à jour des poids
            p.data -= self.lr * self.v[i]

## 🚀 Implémentation d'Adam personnalisé

**Adam** est un optimiseur plus sophistiqué qui :
- Adapte le learning rate pour chaque paramètre individuellement
- Utilise deux moments : **m** (moyenne des gradients) et **v** (variance des gradients)
- Corrige le biais d'initialisation avec des facteurs `m_hat` et `v_hat`

**Avantages** : Converge souvent plus rapidement et de façon plus stable que SGD.

In [ ]:
class AdamPerso:
    def __init__(self, params, lr=1e-3, beta1=0.9, beta2=0.999, eps=1e-8):
        """Initialise l'optimiseur Adam.
        
        Args:
            params: Les paramètres du modèle
            lr: Taux d'apprentissage
            beta1: Coefficient pour le moment d'ordre 1 (moyenne des gradients)
            beta2: Coefficient pour le moment d'ordre 2 (variance des gradients)
            eps: Petit terme pour éviter la division par zéro
        """
        self.params = list(params)
        self.lr = lr
        self.beta1, self.beta2 = beta1, beta2
        self.eps = eps
        self.t = 0  # Compteur d'itérations
        # m: moyenne des gradients, v: variance des gradients
        self.m = [torch.zeros_like(p.data) for p in self.params]
        self.v = [torch.zeros_like(p.data) for p in self.params]

    def zero_grad(self):
        """Remet à zéro tous les gradients."""
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        """Effectue une étape d'optimisation Adam."""
        self.t += 1  # Incrémente le compteur
        for i, p in enumerate(self.params):
            if p.grad is None:
                continue
            g = p.grad.data
            # Mise à jour du moment d'ordre 1 (moyenne mobile exponentielle du gradient)
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g
            # Mise à jour du moment d'ordre 2 (variance)
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (g * g)
            # Correction du biais (important surtout au début de l'entraînement)
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            # Mise à jour des poids avec un learning rate adaptatif
            p.data -= self.lr * m_hat / (torch.sqrt(v_hat) + self.eps)

## 🧠 Définition du réseau de neurones

On crée un **MLP** (Multi-Layer Perceptron) simple avec :
- Une couche d'entrée
- Une couche cachée avec activation ReLU
- Une couche de sortie

C'est un réseau basique mais suffisant pour comparer nos optimiseurs.

In [ ]:
class MLP(nn.Module):
    def __init__(self, d_in, d_hid, d_out):
        """Crée un réseau à 2 couches.
        
        Args:
            d_in: Dimension d'entrée
            d_hid: Nombre de neurones dans la couche cachée
            d_out: Dimension de sortie
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, d_hid),  # Couche linéaire entrée -> cachée
            nn.ReLU(),               # Fonction d'activation non-linéaire
            nn.Linear(d_hid, d_out)  # Couche linéaire cachée -> sortie
        )

    def forward(self, x):
        """Passe avant dans le réseau."""
        return self.net(x)

## 🏋️ Fonction d'entraînement

Cette fonction entraîne notre modèle sur plusieurs époques :
1. **Forward pass** : Calcul de la prédiction
2. **Calcul de la loss** : Erreur entre prédiction et vérité
3. **Backward pass** : Calcul des gradients
4. **Optimisation** : Mise à jour des poids

On utilise la **MSE** (Mean Squared Error) comme fonction de perte.

In [ ]:
def entrainer(model, optim, X, y, epochs=150):
    """Entraîne le modèle et retourne l'historique des pertes.
    
    Args:
        model: Le réseau de neurones
        optim: L'optimiseur (SGD ou Adam)
        X: Données d'entrée
        y: Valeurs cibles
        epochs: Nombre d'époques d'entraînement
    
    Returns:
        Liste des pertes à chaque époque
    """
    loss_fn = nn.MSELoss()  # Fonction de perte MSE
    losses = []
    
    for _ in range(epochs):
        pred = model(X)           # 1. Prédiction
        loss = loss_fn(pred, y)   # 2. Calcul de l'erreur
        optim.zero_grad()         # 3. Réinitialise les gradients
        loss.backward()           # 4. Calcule les gradients
        optim.step()              # 5. Met à jour les poids
        losses.append(loss.item())  # Sauvegarde la perte
    
    return losses

## 🎲 Génération des données synthétiques

On crée un problème de régression linéaire simple :
- `X` : 800 exemples avec 10 features aléatoires
- `y = X × W_true + bruit` : Les vraies valeurs avec un peu de bruit gaussien

Le but du réseau sera d'apprendre à prédire `y` à partir de `X`.

In [ ]:
# Fixe le seed pour la reproductibilité
torch.manual_seed(42)

# Paramètres du dataset
N, D_in, D_out = 800, 10, 1

# Génération des données
X = torch.randn(N, D_in)              # Features aléatoires
W_true = torch.randn(D_in, D_out)     # Poids "vrais" à découvrir
y = X @ W_true + 0.1 * torch.randn(N, D_out)  # Cibles avec bruit

print(f"📊 Dataset créé : {N} exemples, {D_in} features")
print(f"   Shape de X : {X.shape}")
print(f"   Shape de y : {y.shape}")

## 🏃 Entraînement avec SGD

On entraîne un premier modèle avec **SGD + momentum** :
- Learning rate : 0.01
- Momentum : 0.9 (pour accélérer la convergence)
- 200 époques

In [ ]:
hidden = 32   # Nombre de neurones dans la couche cachée
epochs = 200  # Nombre d'époques

# Création du modèle et de l'optimiseur SGD
model_sgd = MLP(D_in, hidden, D_out)
opt_sgd = SGDPerso(model_sgd.parameters(), lr=0.01, momentum=0.9)

# Entraînement
print("🔄 Entraînement avec SGD...")
loss_sgd = entrainer(model_sgd, opt_sgd, X, y, epochs)
print(f"✅ Terminé ! Loss finale : {loss_sgd[-1]:.5f}")

## 🚀 Entraînement avec Adam

On entraîne un second modèle identique mais avec **Adam** :
- Learning rate : 0.01 (même que SGD pour comparer)
- Paramètres beta par défaut (0.9 et 0.999)
- 200 époques

In [ ]:
# Création d'un nouveau modèle identique et de l'optimiseur Adam
model_adam = MLP(D_in, hidden, D_out)
opt_adam = AdamPerso(model_adam.parameters(), lr=0.01)

# Entraînement
print("🔄 Entraînement avec Adam...")
loss_adam = entrainer(model_adam, opt_adam, X, y, epochs)
print(f"✅ Terminé ! Loss finale : {loss_adam[-1]:.5f}")

## 📊 Visualisation des résultats

On compare graphiquement les deux optimiseurs :
- **Graphique 1** (échelle linéaire) : Vue générale de l'évolution de la loss
- **Graphique 2** (échelle log) : Permet de mieux voir les différences en fin de convergence

**À observer** : la vitesse de convergence et la stabilité de chaque méthode.

In [ ]:
plt.figure(figsize=(10, 4))

# Graphique 1 : Échelle linéaire
plt.subplot(1, 2, 1)
plt.plot(loss_sgd, label="SGD", linewidth=2)
plt.plot(loss_adam, label="Adam", linewidth=2)
plt.title("Évolution de la loss", fontsize=14, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, alpha=0.3)

# Graphique 2 : Échelle logarithmique
plt.subplot(1, 2, 2)
plt.semilogy(loss_sgd, label="SGD", linewidth=2)
plt.semilogy(loss_adam, label="Adam", linewidth=2)
plt.title("Convergence (échelle log)", fontsize=14, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss (log)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📈 Résultats finaux et analyse

Comparons les performances finales des deux optimiseurs et analysons les résultats.

In [ ]:
print("="*50)
print("📊 RÉSULTATS FINAUX")
print("="*50)
print(f"Loss finale SGD  : {loss_sgd[-1]:.5f}")
print(f"Loss finale Adam : {loss_adam[-1]:.5f}")
print()

## 🧐 Analyse et conclusions

### Observations clés :

**SGD avec momentum :**
- ✅ Fonctionne bien et converge correctement
- ⚠️ Prend plus de temps à se stabiliser
- 📉 La descente peut être un peu "nerveuse"

**Adam :**
- ✅ Convergence plus fluide, surtout au début
- ✅ Atteint une meilleure loss plus rapidement
- ✅ Plus stable pendant toute la descente

### Conclusion :
Dans cette expérience, les deux optimiseurs arrivent à des résultats finaux proches, mais **Adam** se montre plus efficace :
- Il converge plus vite
- Il est plus stable
- Il nécessite moins de tuning des hyperparamètres

C'est pourquoi Adam est souvent le choix par défaut en Deep Learning ! 🎯

In [ ]:
print("🎓 Analyse détaillée :")
print("-" * 50)
print("SGD :")
print("  • Marche bien mais met du temps à stabiliser")
print("  • Le momentum aide mais reste moins fluide qu'Adam")
print()
print("Adam :")
print("  • Plus fluide dès le début de l'entraînement")
print("  • Atteint une meilleure loss plus rapidement")
print("  • Reste plus stable sur toute la descente")
print()
print("💡 Conclusion : Les deux finissent proches, mais Adam")
print("   converge plus vite et de façon plus stable !")